# Homework 1 - Berkeley STAT 157

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.

In [1]:
import torch

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization.  

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [2]:
A = torch.randn(4096, 4096)
B = torch.randn(4096, 4096)

In [3]:
%%time
C = torch.zeros(4096, 4096)
C = A.mm(B)

CPU times: user 3.83 s, sys: 158 ms, total: 3.98 s
Wall time: 561 ms


In [4]:
%%time
C = torch.zeros(4096, 4096)
for col in range(4096):
    C[:, col] = A.mm(B[:, col].view(-1, 1)).squeeze()

CPU times: user 46.9 s, sys: 96.5 ms, total: 47 s
Wall time: 4.7 s


In [5]:
torchime
C = torch.zeros(4096, 4096)
for row in range(4096):
    for col in range(4096):
        C[row, col] = torch.mm(A[row, :].view(1, -1), B[:, col].view(-1, 1)).item()

KeyboardInterrupt: 

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. 对任意非零实向量$X \in \mathbb{R}^{m \times 1}$,
$$X^T B X = X^T A D A^T X = (A^T X)^T D (A^T X)$$
令$y=A^T X \in \mathbb{R}^{n \times 1}$,则
$$(A^T X)^T D (A^T X) = y^T D y = \sum^n_{i=1}(y_i^2 d_i) \ge 0$$

2. 假设$C \in \mathbb{R}^{m \times c}$, 则$BC$的算法复杂度为$O(m^2c)$,$ADA^TC$的算法复杂度为$O(2mnk+n^2k)$(这个复杂度是怎么算出来的???)。  
    当$m >> n$时，使用$A$和$D$。当$m << n$时，使用$B$。

## 3. PyTorch on GPUs

1. Install GPU drivers (if needed)
1. Install PyTorch on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See [use-gpu](../../chapter_deep-learning-computation/use-gpu.ipynb) for details.

In [6]:
!nvidia-smi

Thu Nov  7 15:17:25 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:B3:00.0 Off |                  N/A |
| 16%   24C    P8     1W / 250W |     26MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
D = torch.ones(2, 2).to('cuda')
D

tensor([[1., 1.],
        [1., 1.]], device='cuda:0')

## 4. Tensor and NumPy 

Your goal is to measure the speed penalty between PyTorch and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in Tensor. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in Tensor for assignments and copy to NumPy at the end.

In [8]:
import numpy as np

A = torch.randn(4096, 4096)
B = torch.randn(4096, 4096)

In [10]:
%time
c = np.zeros(4096)
for i in range(4096):
    c[i] = A.mm(B[:, i].view(-1, 1)).norm().item() ** 2

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 23.4 µs


In [11]:
%time
c = np.zeros(4096)
d = torch.zeros(4096)
for i in range(4096):
    d[i] = A.mm(B[:, i].view(-1, 1)).norm(p=2).item()
c = d.numpy()

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16.5 µs


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [12]:
torch.mm(A, B, out=C)

tensor([[ -29.1079,   16.9853,  -45.4275,  ...,  -78.9186,   72.0830,
           25.4923],
        [  34.5757,   21.6534,   61.2509,  ...,    2.7386,  148.0545,
           17.4609],
        [  74.0823, -125.7534,   93.8716,  ...,  -28.1272,  -10.1727,
           -6.8330],
        ...,
        [  23.3093,  -12.9859, -119.0640,  ...,   71.6363,  146.0223,
          -24.5569],
        [-130.8839,  -41.2710,   61.3858,  ..., -134.8466,  -25.3129,
           16.5158],
        [  93.5614,   -3.1910,   57.2624,  ...,  -31.2232, -113.6387,
          -13.6262]])

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [25]:
x = torch.arange(-10, 10.1, 0.1).view(-1, 1)
j = torch.arange(1, 21, 1).float().view(1, -1)
A = x ** j
A

tensor([[-1.0000e+01,  1.0000e+02, -1.0000e+03,  ...,  1.0000e+18,
         -1.0000e+19,  1.0000e+20],
        [-9.9000e+00,  9.8010e+01, -9.7030e+02,  ...,  8.3451e+17,
         -8.2617e+18,  8.1791e+19],
        [-9.8000e+00,  9.6040e+01, -9.4119e+02,  ...,  6.9514e+17,
         -6.8123e+18,  6.6761e+19],
        ...,
        [ 9.8000e+00,  9.6040e+01,  9.4119e+02,  ...,  6.9514e+17,
          6.8123e+18,  6.6761e+19],
        [ 9.9000e+00,  9.8010e+01,  9.7030e+02,  ...,  8.3451e+17,
          8.2617e+18,  8.1791e+19],
        [ 1.0000e+01,  1.0000e+02,  1.0000e+03,  ...,  1.0000e+18,
          1.0000e+19,  1.0000e+20]])